# Computational problem 2


**The two optmization case studies:**


*   IMAGE RESTORARION
*   Welder beam design


**The three *evolutionary computation* approaches choosed:**


*   Genetic Algorithms
*   Item da lista
*   Differential Evolution

**The three *swarm intelligence* approaches choosed:**

*   particle swarm optimization
*   Item da lista
*   Item da lista

## **Welder beam design**

In [ ]:
def beam_design_objective(h, l, t, b):
    # h, l, t, b = design
    # Ensure the variables are within the constraints
    if h <= 0 or l <= 0 or t <= 0 or b <= 0:
        return -1
    elif t > min(h, b)/2:
        return -1
    # Calculate the moment of inertia
    I = (b*h**3)/12
    # Calculate the maximum stress
    sigma_max = (6*l*I)/(h*b**2)
    # Evaluate the objective function
    objective = 1/sigma_max
    return objective



### **GENETIC ALGORITHM**

In [ ]:
import random
import numpy as np

def objective_function(design):
    """Objective function to optimize beam design"""
    h, l, t, b = design[:4]
    fitness = beam_design_objective(h, l, t, b)
    return fitness

def generate_population(pop_size, design_size):
    """Generate an initial population of designs"""
    low = [0.1, 0.1, 0.1, 0.1]
    high = [2, 10, 10, 2]
    print('size' , pop_size, design_size)
    population = np.random.uniform(low=low, high=high, size=(pop_size, design_size))
    return population


def evaluate_population(population):
    """Evaluate the fitness of each design in the population"""
    fitness = []
    for design in population:
        fitness.append(objective_function(design))
    return fitness

def select_parents(population, fitness, num_parents):
    """Select the best designs to use as parents for crossover"""
    parents = np.empty((num_parents, len(population[0])))
    for parent in range(num_parents):
        max_fitness_idx = np.where(fitness == np.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent, :] = population[max_fitness_idx]
        population = np.delete(population, max_fitness_idx, axis=0)
        fitness = np.delete(fitness, max_fitness_idx)

    return parents

def crossover(parents, offspring_size):
    """Create a new generation of designs by performing crossover"""
    offspring_size = list(offspring_size)
    offspring = np.empty(offspring_size)
    crossover_point = np.uint32(offspring_size[1]/2)
    for k in range(offspring_size[0]):
        parent1_idx = k%parents.shape[0]
        parent2_idx = (k+1)%parents.shape[0]
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

def mutate(offspring):
    """Introduce mutations in the offspring designs"""
    for idx in range(offspring.shape[0]):
        mutation_chance = random.uniform(0, 1)
        if mutation_chance <= 0.1:
            mutation_value = random.uniform(-1, 1)
            offspring[idx, 4] = offspring[idx, 4] + mutation_value
    return offspring

def genetic_algorithm(pop_size, design_size, num_generations):
    """Main function to perform the genetic algorithm"""
    population = generate_population(pop_size, design_size)
    for generation in range(num_generations):
        fitness = np.apply_along_axis(objective_function, 1, population)
        parents = select_parents(population, fitness, pop_size)
        offspring = crossover(parents, (pop_size-parents.shape[0], design_size))
        offspring = mutate(offspring)
        if len(parents) == pop_size:
          population = parents
        else:
          population[0:len(parents), :] = parents
          population[len(parents):, :] = offspring
    best_idx = np.where(fitness == np.max(fitness))
    best_design = population[best_idx[0][0], :]
    return best_design, np.max(fitness)




### **particle swarm optimization**

In [ ]:
import numpy as np

def particle_swarm_optimization(cost_function, num_particles, num_iterations, initial_positions=None,
                                velocity_clamp=None, personal_best_weight=0.5, global_best_weight=0.5):
    """
    Optimize a cost function using Particle Swarm Optimization (PSO) algorithm.

    :param cost_function: The cost function to be minimized.
    :param num_particles: Number of particles in the swarm.
    :param num_iterations: Number of iterations to run the optimization.
    :param initial_positions: Initial positions for each particle. If None, the initial positions are randomly generated.
    :param velocity_clamp: Velocity clamp for each particle. If None, no velocity clamping is performed.
    :param personal_best_weight: Weight applied to the personal best position.
    :param global_best_weight: Weight applied to the global best position.

    :return: The best position found by the swarm and the corresponding cost.
    """
    # Initialize the swarm
    if initial_positions is None:
        positions = np.random.uniform(size=(num_particles, cost_function.dimension))
    else:
        positions = initial_positions

    velocities = np.zeros_like(positions)
    personal_best_positions = positions.copy()
    personal_best_costs = np.array([cost_function.evaluate(p) for p in personal_best_positions])
    global_best_position = personal_best_positions[np.argmin(personal_best_costs)]
    global_best_cost = np.min(personal_best_costs)

    # Run the optimization loop
    for iteration in range(num_iterations):
        # Update the particle velocities
        r1 = np.random.uniform(size=(num_particles, cost_function.dimension))
        r2 = np.random.uniform(size=(num_particles, cost_function.dimension))
        velocities = (personal_best_weight * r1 * (personal_best_positions - positions) +
                      global_best_weight * r2 * (global_best_position - positions))

        # Apply velocity clamping, if specified
        if velocity_clamp is not None:
            velocities = np.clip(velocities, -velocity_clamp, velocity_clamp)

        # Update the particle positions
        positions = positions + velocities

        # Update the personal best positions and costs
        for i in range(num_particles):
            cost = cost_function.evaluate(positions[i])
            if cost < personal_best_costs[i]:
                personal_best_positions[i] = positions[i].copy()
                personal_best_costs[i] = cost

                # Update the global best position and cost
                if cost < global_best_cost:
                    global_best_position = positions[i].copy()
                    global_best_cost = cost

    return global_best_position, global_best_cost


In [ ]:
pop_size = 1000
design_size = 4
num_generations = 500

best_design, best_fitness = genetic_algorithm(pop_size, design_size, num_generations)
all = genetic_algorithm(pop_size, design_size, num_generations)

#h (height), l (length), t (thickness), and b (width):

print("Best design: ", best_design)
print("Best fitness: ", best_fitness)
print("all" , all)

size 1000 4
size 1000 4
Best design:  [1.03177267 0.32352303 0.13013891 1.17335962]
Best fitness:  6.813776762715063
all (array([0.56370355, 0.26430367, 0.19024703, 1.1368552 ]), 27.072628186601364)


### **Util functions**

In [ ]:
def summarize_objective_function_values(objective_function_values):
    min_value = np.min(objective_function_values)
    max_value = np.max(objective_function_values)
    mean_value = np.mean(objective_function_values)
    median_value = np.median(objective_function_values)
    std_value = np.std(objective_function_values)
    
    return (min_value, max_value, mean_value, median_value, std_value)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_convergence(fitness_values):
    plt.figure(figsize=(10, 5))
    plt.plot(range(len(fitness_values)), np.log10(fitness_values[:, 0]), label="Min Fitness Value")
    plt.plot(range(len(fitness_values)), np.log10(fitness_values[:, 1]), label="Max Fitness Value")
    plt.plot(range(len(fitness_values)), np.log10(fitness_values[:, 2]), label="Mean Fitness Value")
    plt.xlabel("Iteration")
    plt.ylabel("Log 10 of Fitness Value")
    plt.legend()
    plt.show()

### **Image restoration application**

In [ ]:
# GENETIC ALGO APPLY TO IMAGE RESTORATION

import numpy as np
import random
import deap
from deap import base, creator, tools

# Define the evaluation function for the optimization problem
def eval_func(individual):
    # Calculate the fitness value based on the image restoration problem
    fitness = ...
    return fitness,

# Define the problem-specific parameters
IMAGE_WIDTH = ...
IMAGE_HEIGHT = ...
POPULATION_SIZE = ...
NUM_GENERATIONS = ...

# Create a DEAP optimization problem
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Initialize the toolbox
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, IMAGE_WIDTH * IMAGE_HEIGHT)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define the genetic operators
toolbox.register("evaluate", eval_func)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Start the optimization process
population = toolbox.population(n=POPULATION_SIZE)
NGEN = NUM_GENERATIONS
for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))

best_individual = tools.selBest(population, k=1)[0]


One swarm intelligence approach that can be used for image restoration is Particle Swarm Optimization (PSO). PSO is a population-based optimization algorithm that is inspired by the social behavior of birds flocking or fish schooling. The basic idea of PSO is to maintain a set of candidate solutions, called particles, which fly through the solution space and update their positions based on their own experience and the experience of other particles. The particles are moved towards the best solution found so far by the swarm, which is referred to as the global best solution.

Here is a simple implementation of PSO for image restoration in Python:

In [ ]:
import numpy as np
import cv2

def restore_image(image):
    # read the image and convert it to grayscale
    img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)

    # define the number of particles and iterations
    num_particles = 100
    num_iterations = 1000

    # initialize the particles
    particles = np.random.rand(num_particles, img.shape[0] * img.shape[1])
    particles = particles * 255
    particles = particles.astype(np.uint8)

    # initialize the velocity and best position of each particle
    velocity = np.zeros(particles.shape)
    best_particle_pos = particles.copy()
    best_particle_fitness = np.zeros(num_particles)

    # initialize the global best particle
    global_best_particle = particles[0]
    global_best_fitness = np.inf

    # set the weight, acceleration coefficients and random values
    weight = 0.729
    c1 = 1.49445
    c2 = 1.49445
    r1 = np.random.rand(num_particles)
    r2 = np.random.rand(num_particles)

    # iterate for the specified number of iterations
    for i in range(num_iterations):
        # evaluate the fitness of each particle
        for j in range(num_particles):
            fitness = np.sum(np.abs(img - particles[j].reshape(img.shape)))
            if fitness < best_particle_fitness[j]:
                best_particle_fitness[j] = fitness
                best_particle_pos[j] = particles[j].copy()
            if fitness < global_best_fitness:
                global_best_fitness = fitness
                global_best_particle = particles[j].copy()

        # update the velocity and position of each particle
        for j in range(num_particles):
            velocity[j] = weight * velocity[j] + c1 * r1[j] * (best_particle_pos[j] - particles[j]) + c2 * r2[j] * (global_best_particle - particles[j])
            particles[j] = particles[j] + velocity[j]

    # reshape the global best particle to the original shape of the image
    restored_img = global_best_particle.reshape(img.shape)

    return restored_img

image_file = '/content/test.jpg'
restored_img = restore_image(image_file)    


### Tabela com os resultados

In [ ]:
import pandas as pd
function_names = ['rosenbrock', 'rosenbrock_2', 'beale', 'mathias', 'ackley', 'booth', 'rastrigin_2d', 'rastrigin_n']

methods = ['BFGS', 'L-BFGS-B', 'Powell']
metrics = ['max', 'min', 'mean', 'median']

data = []
for function in table_values:
    for func_name, func_data in function.items():
        for method in methods:
            row = [func_name, method]
            for metric in metrics:
                row.append(func_data[method][metric])
            data.append(row)

df = pd.DataFrame(data, columns=['Function', 'Method', 'Max', 'Min', 'Mean', 'Median'])

display(df.style.set_caption("Comparison of Optimization Methods on Test Functions").set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f2f2f2'), ('color', '#333'), ('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('text-align', 'center'), ('font-size', '14px')]}
]))